In [1]:
# import libraries
import requests # send http requests
import re # regex to deal with strings
from bs4 import BeautifulSoup # pull data from html and xml files
# import urllib.request # open urls - good for downloading files

In [2]:
# semester to be extracted: the format is Fall/Spring/Summer + YYYY. If this does not work, inspect the HTML
semester = 'Spring2020'
semesterURL = semester + '.html' + '$' # add $ at the end to specify that the link ends with semester.html

In [3]:
# extract the URLs for all subjects
# subjects are grouped by alphabets, so we create a alphabetical list for the loop
import string
alphabet = list(string.ascii_uppercase) 

subjList = [] # initialize the list of URLs
for letter in alphabet:
    letterURL = 'http://www.columbia.edu/cu/bulletin/uwb/sel/subj-' + letter + '.html' #link for subjects beginning with the letter
    html = requests.get(letterURL)
    soup = BeautifulSoup(html.text, "html.parser") # parse the html file
    
    subjLetter = soup.findAll('a', href = re.compile(semesterURL)) # get the attributes containing the links
    for subj in subjLetter:
        subjList.append('http://www.columbia.edu' + subj['href']) # add links to list
        # a sample URL: 'http://www.columbia.edu/cu/bulletin/uwb/subj/ACCT/_Spring2020.html'
        
print('Number of Subjects in ' + semester + ' is:', len(subjList))

Number of Subjects in Spring2020 is: 321


In [ ]:
subjData = [] # initialize data
for subj in subjList:
    html = requests.get(subj)
    soup = BeautifulSoup(html.text, "html.parser")

    # extract class info from a table in the html where info is rows
    classTable = soup.findAll('tr') # parse all rows
    # drop row 0,1 and last row which contain irrelevant information
    del classTable[0:2]
    del classTable[-1]
    
    # for each class, the info is extracted as follows:
    # 1. class code and name: the row with only 1 cell
    # 2. all other info including enrollment, instructor, time, location: the row with 2 cells
    for cl in classTable:
        row = {} # initialize row
        if len(cl.findChildren('td')) == 1:
            row['code'] = cl.findChildren('td')[0].b.next_element[-5:] # code comes in 5 characters at the end of the first line
            row['name'] = cl.findChildren('td')[0].b.next_element.next_element.next_element # name comes in the second line
        else:
            info = cl.findChildren('td')[1].findAll('b')
            for elm in info:
                row[elm.text.replace(':','').strip()] = elm.next_element.next_element.strip() # strip to remove spaces in front and behind
        subjData.append(row)

In [ ]:
# store data in a data frame
import pandas as pd
df = pd.DataFrame(subjData)

In [ ]:
# some cleaning for readability in excel
df = df.fillna(0)

# move the code and name info on the first row with all other info to signal the start of a class
df['Code'] = df.loc[df['Call Number'].shift(-1) != 0, 'code']
df['Code'] = df['Code'].shift()
df['Name'] = df.loc[df['Call Number'].shift(-1) != 0, 'name']
df['Name'] = df['Name'].shift()

# merge the 2 columns on instructors
df['Instructors'] = df['Instructor'].astype(str) + ',' + df['Instructors'].astype(str)
df['Instructors'] = df['Instructors'].str.replace(',0','',regex=True)
df['Instructors'] = df['Instructors'].str.replace('0,','',regex=True)

# remove the (code,name,Instructor) columns and remove the row with only the code and name info
df = df.drop(columns=['code','name', 'Instructor'],axis=1)
df = df.drop(df[df['Call Number'] == 0].index)

# extract the current enrollment
import numpy as np
df['Size'] = df['Enrollment'].str[:3]
df['Size'] = np.where(df['Size'].str[2] == 's', df['Size'].str[0], df['Size']) #if the 3rd character is a 's', size is single-digit
df['Size'] = np.where(df['Size'].str[3] == 's', df['Size'].str[:1], df['Size']) #if the 4th character is a 's', size is double-digit
df['Size'] = df['Size'].astype(int)

# re-order the variables and take only what we need
columns = ['Name','Code','Call Number','Enrollment','Size','Day/Time','Location','Instructors']
df = df[columns]
df = df.replace(0,'')

In [ ]:
# save to excel
fileName = 'class' + semester + '.xlsx'
df.to_excel(fileName)